Setup Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from collections import OrderedDict
import h5py
import importlib
DataDir = '/nfs/Data/Proteasome Project Data'
import sys
sys.path.append(DataDir)
import matplotlib.lines as mlines


In [3]:
from wand.image import Image as WImage
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

Import Data Function

In [4]:
N = 1
def ChangeDS(NewWorkingName):
    global N
    global WorkingName
    global FullOutArray
    global Errors
    global HasData
    global Runtimes
    global A0Range
    global KD1Range
    global KD2Range
    global Kp
    global TimePoints
    global SpeciesCount
    global Delta
    global Deg
    
    WorkingName = NewWorkingName
    
    try:
        del FullOutArray
    except:
        pass

    File = h5py.File("{}/{}{}.h5".format(DataDir, N, WorkingName), "r")
    if N == 1:
        CD = importlib.import_module("{}{}".format("",WorkingName))
    else:
        CD = importlib.import_module("{}{}".format(N-1,WorkingName))

    FullOutArray = File["Results"][:]
    FullOutArray = FullOutArray[::-1,::-1,:,:,:]


    Errors = File["Errors"][:]
    HasData = File["HasData"][:]
    Runtimes = File["Runtimes"][:]
    A0Range = File["A0Range"][:]
    Delta = CD.Delta
    Deg = Delta

    KD1Range = File["KD1Range"][:]
    KD1Range = KD1Range[::-1]

    KD2Range = File["KD2Range"][:]
    KD2Range = KD2Range[::-1]

    Kp = File["Kp"][()]
    TimePoints = CD.KeepTimePoints
    SpeciesCount = CD.SpeciesCount
    print "Working Name: {}".format(WorkingName)
    print "Shape: {}".format(Errors.shape)
    print "Errors: {}".format(np.sum(Errors==True))
    print "HasData: {}".format(np.sum(HasData==True))

In [5]:
import os, fnmatch
DataSets = []
listOfFiles = os.listdir(DataDir)
pattern = "STV3*.py"
for entry in listOfFiles:  
    if fnmatch.fnmatch(entry, pattern):
        DataSets.append(entry[:-3])
print (DataSets)

['STV3SynDeg17e-05T', 'STV3SynDeg28e-04T', 'STV3SynDeg10e-09T', 'STV3StaticT', 'STV3SynDeg32e-08T', 'STV3SynDeg10e-16T', 'STV3SynDeg10e-11T', 'STV3SynDeg11e-06T', 'STV3SynDeg27e-07T', 'STV3SynDeg43e-06T', 'STV3SynDeg35e-05T', 'STV3SynDeg10e-10T', 'STV3SynDeg22e-06T', 'STV3SynDeg87e-06T', 'STV3SynDeg10e-14T', 'STV3SynDeg10e-13T', 'STV3SynDeg10e-12T', 'STV3SynDeg54e-07T', 'STV3SynDeg10e-15T', 'STV3SynDeg69e-05T', 'STV3SynDeg56e-04T', 'STV3SynDeg14e-04T']


In [6]:
# DSName = "STV2StaticT"
DSName = "STV2SynDeg28e-04T"
ChangeDS(DSName)
# ChangeDS("STV2SynDeg28e-04T")
if "Static" in DSName:
    CDataSet = "in vitro-like"
else:
    CDataSet = "in vivo-like"

Working Name: STV2SynDeg28e-04T
Shape: (100, 100)
Errors: 0
HasData: 1344000000


In [7]:
# ColorList = {
#     "SW":"#d7191c",
#     "WW":"#fdae61",
#     "SS":"#ffffbf",
#     "WS":"#abd9e9",
#     "M":"#2c7bb6",
# }
# ColorList = {
#     "SW":"#ca0020",
#     "WW":"#f4a582",
#     "SS":"#f7f7f7",
#     "WS":"#92c5de",
#     "M":"#0571b0",
# }
# ColorList = {
#     "SW":"",
#     "WW":"",
#     "SS":"",
#     "WS":"",
#     "M":"",
# }
ColorList = {
    "SW":"#e41a1c",
    "WW":"#377eb8",
    "SS":"#4daf4a",
    "WS":"#984ea3",
    "M":"#ff7f00",
}
# ColorList = {
#     "SW":"#ca0020",
#     "WW":"#f4a582",
#     "SS":"#f7f7f7",
#     "WS":"#92c5de",
#     "M":"#0571b0",
# }

In [8]:
DataLocations = {
    "SW":(np.min(KD1Range), np.max(KD2Range)),
    "WW":(np.max(KD1Range), np.max(KD2Range)),
    "SS":(np.min(KD1Range), np.min(KD2Range)),
    "WS":(np.max(KD1Range), np.min(KD2Range)),
    "M":(KD1Range[55], KD2Range[55]),
}
DataIndices = {
    "SW":(-1, 0),
    "WW":(-1, -1),
    "SS":(0, 0),
    "WS":(0, -1),
    "M":(55, 55),
}

DataNames = {x:map("{:.0e}".format,y) for (x,y) in DataLocations.items()}

In [9]:
# Add Binary Asm Pathways

import AsmPathsBinary as AB
D, SortedPathways = AB.AsmPathways()
P = AB.Probabilities(D) 
# -----------------------------------------------------------------------------------------
def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return a.flat[idx]

In [10]:
ShowPlots = False

A0 = 4.0e-6
# A0 = 1.0e-8
# Time = 1.0e8
Time = 3600*24 # One Day

# Setup Initial Derived Values
TimeTarget = np.argmax(TimePoints==find_nearest(TimePoints, Time))
A0Target = np.argmax(A0Range==find_nearest(A0Range, A0))
A0 = A0Range[A0Target]
Time = TimePoints[TimeTarget]
print Time
print A0

75646.3327555
4.32876128108e-06


In [11]:
np.save("./FigureData/Contour-Data-{}.npy".format(CDataSet),FullOutArray[:,:,A0Target,TimeTarget,11].T)

In [12]:
if len(A0Range) > 5:
    A0Start = np.argmax(A0Range > 1e-9)-1
    A0Stop = np.argmax(A0Range > 1e-5)
else:
    A0Start = 0
    A0Stop = len(A0Range)

ConcRange = A0Range[A0Start:A0Stop]

FinalFlux = np.zeros(len(SortedPathways.values()))
Flux3D = np.zeros([len(ConcRange),len(SortedPathways.values())])

In [13]:
AsmDataSets1D = dict()
PrepFrac = FullOutArray[::-1,::-1,:,TimeTarget,:]
for DataSet in DataIndices:
    Frac = PrepFrac[DataIndices[DataSet][0],DataIndices[DataSet][1],:,:]
    Flux = np.zeros(len(SortedPathways.values()))
    FluxByA0 = np.zeros([len(A0Range),len(SortedPathways.values())])
    for i, CA0 in enumerate(A0Range):
        P.UpdateConcentrations(Frac[i,:], CA0)
        for k, Path in enumerate(SortedPathways.values()):
            T = P.PathIndependentFlux(Path, Probability = True)
            Flux[k] = T
        FluxByA0[i] = Flux
    Mean = np.mean(FluxByA0, 0)
    Min  = np.mean(FluxByA0, 0) - np.min(FluxByA0, 0)
    Max  = np.max(FluxByA0, 0) - np.mean(FluxByA0, 0)
#     Mean[20] = 0.5
#     Min[20] = 0.4
#     Max[20] = 0.2
    AsmDataSets1D[DataSet] = (Mean, Min, Max)
    
#     for k, Path in enumerate(SortedPathways.values()):
#         T = P.PathIndependentFlux(Path, Probability = True)
#         FinalFlux[k] = T
#     FluxIndex = range(1, len(FinalFlux)+1)
#     Average = []
#     StdDev  = []
#     for i in FluxIndex:
#         Average.append(np.mean(Flux3D[:,i-1]))
#         StdDev.append(np.std(Flux3D[:,i-1]))

AddLineFracVs(
                MainPlot,
                PlotIndex,
                DataSet,
                Width = MP.inches2mm(3.25),
                FontSize = 5.5,
                Aspect= 4.0/3.0,
                hpad=0.2,
                wpad=0.40,
                ShiftRight = 0.0,
                TimeOrConc = "Conc" # "Conc" or "Time"
            ):

In [14]:
# from MPLGUI import ContourPlot
import MPLGUI as MP
import matplotlib.gridspec as gridspec
__author__ = 'Koan'

def AddLineFracVs(
                MainPlot,
                PlotIndex,
                DataSet,
                Width = MP.inches2mm(3.25),
                FontSize = 5.5,
                Aspect= 4.0/3.0,
                hpad=0.2,
                wpad=0.40,
                ShiftRight = 0.0,
                TimeOrConc = "Conc" # "Conc" or "Time"
            ):
    import numpy as np
    import MPLGUI as MP
    from matplotlib.offsetbox import AnnotationBbox, OffsetImage

    YAxis = FullOutArray[::-1,::-1,:,:,11]
    XMin = 0.0
    XMax = 0.0
    YMin = 0.0
    YMax = 0.0
    LineSet=[]
    for DataSet, Key in DataIndices.items():
        if TimeOrConc == "Conc":
            TitleText = "Initial Monomer Concentration (M)"
            CData = YAxis[Key[0],Key[1],:, TimeTarget]
            XAxis = A0Range
            if np.max(CData) > YMax:
                YMax = np.max(CData)
            if np.max(CData) < YMin:
                YMin = np.min(CData)

        elif TimeOrConc == "Time":
            TitleText = "Time (s)"
            CData = YAxis[Key[0],Key[1],A0Target, :]
            XAxis = TimePoints
            if np.max(CData) > YMax:
                YMax = np.max(CData)
            if np.max(CData) < YMin:
                YMin = np.min(CData)
        LineSet.append((DataSet, CData))
        
    XLim = [np.min(XAxis),np.max(XAxis)]

    if MainPlot:
        MF = MainPlot._MainFig
    else:
        MF = MainPlot
    
    MainPlot = MP.LinePlot(
        YAxisInPercent = False,
        Width=Width,
        PrimaryFig=MF,
        subplot=PlotIndex,
        Aspect=Aspect,
        hpad = hpad,
        wpad = wpad,
#         axes_aspect=axes_aspect,
    )

    for Line in LineSet:
        MainPlot.AddLine(
            XAxis,
            Line[1],
            xlog = True,
            label=Line[0],
            color=ColorList[Line[0]],
            linewidth=3,
            alpha=0.8,
        )
#     MainPlot.AddLegend()

    MainPlot._Plot.tick_params(
            which = 'major',
            labelsize=FontSize*0.8,
            pad = 1.5, # How far are the labels from the mark
    )

    MainPlot.XLabel(TitleText)
    MainPlot.YLabel("Yield of Maximal Structure")
#     MainPlot.SetXRange(XMin,XMax)
    MainPlot.SetYRange(YMin,YMax*1.05)

#         MainPlot.SaveFig(ImgFileName, bbox_inches = 'tight', antialiased=True)
#         MP.OpenPDF(ImgFileName)
#     l,b,w,h = MainPlot.gca().get_position().bounds

#     print plt.gca().get_position().bounds
# #         ll,bb,ww,hh = Colorbar.ax.get_position().bounds
#     print Colorbar.ax.get_position().bounds
# #         Colorbar.ax.set_position([ll, b+0.1*h, ww, h*0.8])
#     Colorbar.ax.set_position(np.array(Colorbar.ax.get_position().bounds) + [ShiftRight,0.0,0,0])
#     MainPlot._Plot.set_position(np.array(MainPlot._Plot.get_position().bounds) + [ShiftRight,0.0,0,0])

    MainPlot._Plot.set_title(CDataSet, style='italic')
    XTicks = MainPlot._Plot.xaxis.get_major_ticks()
    for Index in np.arange(0, len(XTicks),2):
        XTicks[Index].label1On = ""

    print "Done"
    return MainPlot

def AddContour(
                MainPlot,
                PlotIndex,
                DataSet,
                Width = MP.inches2mm(3.25),
                FontSize = 5.5,
                Aspect=1.0,
                hpad=0.3,
                wpad=0.3,
                axes_aspect="equal",
                ShiftRight = 0.0
            ):

In [15]:
# from MPLGUI import ContourPlot
import MPLGUI as MP
import matplotlib.gridspec as gridspec
__author__ = 'Koan'

def AddContour(
                MainPlot,
                PlotIndex,
                DataSet,
                Width = MP.inches2mm(3.25),
                FontSize = 5.5,
                Aspect=1.0,
                hpad=0.3,
                wpad=0.3,
                axes_aspect="equal",
                ShiftRight = 0.0
            ):
    import numpy as np
    import MPLGUI as MP
    from matplotlib.offsetbox import AnnotationBbox, OffsetImage

    Images = MP.ImportImages()

    IP = False
    InVivo = False
    ContourData = FullOutArray[:,:,A0Target,TimeTarget,11].T

#     KD1Range = np.load("{}/{}".format(MP.DefaultDataDir, "KDRange.npy"))
#     KD2Range = np.load("{}/{}".format(MP.DefaultDataDir, "KDRange.npy"))

    XLim = [np.max(KD1Range),np.min(KD1Range)]
    YLim = [np.max(KD2Range),np.min(KD2Range)]

    XBounds = (XLim[0] <= KD1Range) & (KD1Range<= XLim[1])
    XBounds = (YLim[0] <= KD2Range) & (KD2Range<= YLim[1])

    if MainPlot:
        MF = MainPlot._MainFig
    else:
        MF = MainPlot
    gs1 = gridspec.GridSpec(nrows=3, ncols=3, left=0.05, right=0.48, wspace=0.05)
    MainPlot = MP.ContourPlot(
        Width=Width,
        PrimaryFig=MF,
        subplot=PlotIndex,
        Aspect=Aspect,
        hpad = hpad,
        wpad = wpad,
        axes_aspect=axes_aspect,
    )
    """
    gs = gridspec.GridSpec(3,2)
    global T1
    T1 = gs
    global T2
    ll,bb,ww,hh = gs.get_position().bounds
    CB.ax.set_position([ll, bb, ww, hh])
#     MainPlot._Plot.set_position(gs[0:2].get_position(MainPlot._MainFig))
#     MainPlot._Plot.set_subplotspec(gs[0:2]) 
    """

    XTicks = MainPlot._Plot.xaxis.get_major_ticks()
    YTicks = MainPlot._Plot.yaxis.get_major_ticks()
    for Index in np.arange(0, len(XTicks),2):
        XTicks[Index].label1On = ""
        YTicks[Index].label1On = ""

    import matplotlib.cm as cm
    
#     ColorMap = cm.gray_r
#     ColorMap = cm.gray
    ColorMap = cm.binary
#     ColorMap = cm.plasma_r
    
    ContourImage = MainPlot._Plot.pcolormesh(
        KD1Range,
        KD2Range,
        ContourData,
        cmap=ColorMap,
        shading="gouraud"
        # vmin=0.0,
        # vmax=1.0,
    )
    ContourImage.autoscale()
    ZLim = [np.min(ContourData), np.max(ContourData)]
#         ZLim = [0.0,1.0]
    Delta = ZLim[1] - ZLim[0]
    ContourLevels = [
            ZLim[0]+1.0*Delta/5.0,
            ZLim[0]+2.0*Delta/5.0,
            ZLim[0]+3.0*Delta/5.0,
            ZLim[0]+4.0*Delta/5.0,
    ]

#         if DataSet == "In Vivo":
#             ContourLevels = [
#                     ZLim[0]+1.0*Delta/5.0,
#                     ZLim[0]+2.0*Delta/5.0,
#                     ZLim[0]+3.0*Delta/5.0,
#                     ZLim[0]+4.0*Delta/5.0,
#                     ]
# #             ContourLevels = [
# #                     0.40,
# #                     0.60,
# #                     0.63,
# #                     0.65,
# #                     ]
#         elif DataSet == "In Vitro":
#             ContourLevels = [
#                     0.50,
#                     0.59,
#                     0.75,
#                     0.99,
#                     ]
    print ZLim
    print ContourLevels
    CLs = MainPlot._Plot.contour(
        KD1Range,
        KD2Range,
        ContourData,
        ContourLevels,
        origin='lower',
        linewidths=1.0,
        colors=(
            'red',
            'blue',
            'green',
            'purple',
        )
    )
    if ContourLevels[-1] > 0.95:
        ContourLevels = ContourLevels[:-1]
    Colorbar = MP.mpl.colorbar(
            ContourImage,
            #     ticklocation = [],
            ticks = [ZLim[0],ZLim[1]]+ContourLevels,
            format = '%1.1f',
#                 format = '%3d%%',
            drawedges = False,
            orientation = 'vertical',
            #     shrink=0.8,
            #     extend='neither'
#                 shrink=0.8
        )

    MainPlot._Plot.tick_params(
            which = 'major',
            labelsize=FontSize*0.8,
            pad = 1.5, # How far are the labels from the mark
    )
    Colorbar.ax.tick_params(which='both',length=0, labelsize=FontSize*0.8)
    # Colorbar.ax.set_xticks([ZLim[0],ZLim[1]]+ContourLevels)
    Colorbar.add_lines(CLs)
    # Colorbar.set_ticks(ContourLevels)
    Colorbar.update_ticks()

#         MainPlot._Plot.clabel(
#             ContourLines,
#             ContourLevels,
#             inline=1,
#             fmt='%.2f',
#             fontsize=7,
#             orientation='vertical',
#             inline_spacing = 30.0,
#         )

#     MainPlot.SaveFig(ImgFileName, bbox_inches = 'tight')

    MarkerSize = 10
    for Key in DataLocations.keys():
        CMS = MarkerSize
        if DataLocations[Key][1] < 9e-11 or DataLocations[Key][1] > 9e-4 or DataLocations[Key][0] < 9e-11 or DataLocations[Key][0] > 9e-4:
            CMS = 30
        MainPlot._Plot.loglog(
            DataLocations[Key][1],DataLocations[Key][0],
            marker="o",
            markersize = CMS,
            color=ColorList[Key],
        )

    MainPlot.XLabel(r"Intra-Ring K$_d$ (M)")
    MainPlot.YLabel(r"Inter-Ring K$_d$ (M)")
    MainPlot.SetXRange(XLim[0],XLim[1])
    MainPlot.SetYRange(YLim[0],YLim[1])

#         MainPlot.SaveFig(ImgFileName, bbox_inches = 'tight', antialiased=True)
#         MP.OpenPDF(ImgFileName)
#     l,b,w,h = MainPlot.gca().get_position().bounds

#     print plt.gca().get_position().bounds
# #         ll,bb,ww,hh = Colorbar.ax.get_position().bounds
#     print Colorbar.ax.get_position().bounds
# #         Colorbar.ax.set_position([ll, b+0.1*h, ww, h*0.8])
    Colorbar.ax.set_position(np.array(Colorbar.ax.get_position().bounds) + [ShiftRight,0.0,0,0])
    MainPlot._Plot.set_position(np.array(MainPlot._Plot.get_position().bounds) + [ShiftRight,0.0,0,0])

    MainPlot._Plot.set_title(DataSet, style='italic')

    print "Done"
    return MainPlot

AddPlot(MainPlot,
            PlotIndex,
            DataSet,
            Aspect= 4.0/3.0,
            hpad=0.2,
            wpad=0.40,
            FontSize=5.5,
           ):

In [16]:
__author__ = 'Koan'

from MPLGUI import *
import numpy as np

def AddPlot(MainPlot,
            PlotIndex,
            DataSet,
            Aspect= 4.0/3.0,
            hpad=0.2,
            wpad=0.40,
            FontSize=5.5,
           ):
    
    
    MainPlot = RankWBinaryImages(
        Width = inches2mm(3.25),
        subplot=PlotIndex,
        PrimaryFig=MainPlot._MainFig,
        Aspect=Aspect,
        hpad = hpad,
        wpad = wpad,
        FontSize=FontSize,
    )
    MainPlot.SetPathwayData(
        AsmDataSets1D[DataSet][0],
        Min = AsmDataSets1D[DataSet][1],
        Max = AsmDataSets1D[DataSet][2],
        )

    #  0 :    "32  1 0 0.0 0 0.png"
    #  1 :     "48  1 1 0.0 0 0.png"
    #  2 :     "36  1 0 0.1 0 0.png"
    #  3 :     "56  1 1 1.0 0 0.png"
    #  4 :     "52  1 1 0.1 0 0.png"
    #  5 :     "50  1 1 0.0 1 0.png"
    #  6 :     "60  1 1 1.1 0 0.png"
    #  7 :     "53  1 1 0.1 0 1.png"
    #  8 :     "54  1 1 0.1 1 0.png"
    #  9 :     "51  1 1 0.0 1 1.png"
    # 10 :     "62  1 1 1.1 1 0.png"
    # 11 :     "63  1 1 1.1 1 1.png"

    ImageOrderTop = [
        3,4,5,6,8,10,
        ]
    ImageOrderBottom = [
        3,4,5,1,2,0,
        ]
    Images = ImportImages()
    TopSeqStartHeight = MainPlot.TopSeqStartHeight
    StepWidth = 46.0/6.0
    #ImageZoom = 0.09
#     ImageZoom = 0.045
    ImageZoom = 0.055
#     ImageZoom = 0.05
#     ImageZoom = 0.02
    XOffset = 0.15
    TopRowPos        =    0.75 + TopSeqStartHeight
#     TopRowPos        =    1.95
    # BottomRowPos    =    1.45
    BottomRowPos    =    0.225 + TopSeqStartHeight
    for Index in np.arange(1.0,7.0):
        MainPlot._Plot.plot(
            StepWidth*Index-StepWidth/2.0+XOffset,
            (TopRowPos+BottomRowPos)/2.0,
            '+',
            color    =    "black",
            mew        =    1.5,
            ms        =    10.0,
        )
        xy = [StepWidth*Index-StepWidth/2.0+XOffset, TopRowPos]
        ab = AnnotationBbox(
            OffsetImage(
                Images[ImageOrderTop[int(Index-1)]],
                zoom = ImageZoom,
                resample = True,
                dpi_cor = False,
                # Interpolation Options : 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'
                interpolation = "gaussian",
                ),
            xy,
            xybox=(1.0, 1.0),
            xycoords='data',
            boxcoords="offset points",
            pad=0.0,
            frameon = False,

            )
        MainPlot._Plot.add_artist(ab)

    for Index in np.arange(1.0,7.0):
        xy = [StepWidth*Index-StepWidth/2.0+XOffset, BottomRowPos]
        ab = AnnotationBbox(
            OffsetImage(
                Images[ImageOrderBottom[int(Index-1)]],
                zoom = ImageZoom,
                resample = True,
                dpi_cor = False,
                # Interpolation Options : 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'
                interpolation = "gaussian",
                ),
            xy,
            xybox=(1.0, 1.0),
            xycoords='data',
            boxcoords="offset points",
            pad=0.0,
            frameon = False,

            )
        MainPlot._Plot.add_artist(ab)
        
        MainPlot._Plot.tick_params(
#                 which = 'major',
                labelsize=FontSize*0.8,
#                 pad = 1.5, # How far are the labels from the mark
        )


    #mpl.tight_layout(rect = [0, 0, 0.4, 1])
    ImgFileName = "LargeHist-{}.pdf".format(DataSet)

    # MainPlot.SaveFig(
    #     ImgFileName,
    #     bbox_inches = 'tight',
    #     antialiased=True
    # )
#     c = mpatches.Circle(
#         5.0,
#         facecolor="green",
#         label="Hi"
#         )
#     plt.gca().add_patch(c)
    
#     Title = mpatches.Patch(color='red', label='The red data')
    Title = mlines.Line2D([], [],
                          color=ColorList[DataSet],
                          marker='o',
                          linestyle='None',
                          markersize=10,
                         )
    plt.legend(
        [Title],
        [r"Inter-Ring K$_d$ $10^{{{:.0f}}}$ (M) : Intra-Ring K$_d$ $10^{{{:.0f}}}$ (M)".format(*np.log10(DataLocations[DataSet]))],
        numpoints=1,
        bbox_to_anchor=(0., 1.0, 1., .08),
        loc=9,
        ncol=1,
#         mode="expand",
        borderaxespad=0.,
        frameon=False,
        fontsize=FontSize,
    )
    print "Done"
    return MainPlot

In [17]:
import MPLGUI as MP
from wand.image import Image as WImage

try:
    del img
except:
    pass

FontSize = 11

DataSet = "WW"

Settings = {"hpad": 0.3, "wpad": 0.2, "FontSize": FontSize}

# MainPlot = RankWBinaryImages(FontSize = 6.5, Width=inches2mm(4), Aspect=(4.0/3.0))
Subplot = 321
OverallMainPlot = MPLGPU(
    Blank=True,
#     hpad=0.1,
#     wpad=0.1,
#     FontSize = FontSize,
    Width=MP.inches2mm(6.5),
    subplot=321,
#     axes_aspect="equal",
    Aspect=16.0/12.0,
    axes_aspect=16.0/12.0,
    **Settings
)

gs1 = gridspec.GridSpec(nrows=3, ncols=2)#, left=0.05, right=0.48, wspace=0.05)

AddPlot(OverallMainPlot, gs1[0,0], "SW", **Settings)
AddPlot(OverallMainPlot, gs1[0,1], "SS", **Settings)
AddPlot(OverallMainPlot, gs1[1,1], "M", **Settings)
AddPlot(OverallMainPlot, gs1[2,0], "WW", **Settings)
AddPlot(OverallMainPlot, gs1[2,1], "WS", **Settings)


#gs2 = gridspec.GridSpec(nrows=3, ncols=2, left=0.0, right=0.48, wspace=0.0)
# gs1.update(left=0.1, right=3.25)
CP = AddContour(
    OverallMainPlot,
    gs1[1,0],
    CDataSet,
    Width = MP.inches2mm(3.5),
    Aspect=1.0,
    axes_aspect="equal",
    ShiftRight = 0.025,
    **Settings
    )
# OverallMainPlot._MainFig.subplots_adjust(
#     left=None,
#     bottom=None,
#     right=None,
#     top=None,
#     wspace=0.4,
#     hspace=0.35
#     )

ImgFileName = "LargeHist-Multi-{}.pdf".format(CDataSet)
ImgFileName = "".join(ImgFileName.split())
OverallMainPlot.SaveFig(
    ImgFileName,
    bbox_inches = 'tight',
    antialiased=True
)
del OverallMainPlot

# OverallMainPlot = MPLGPU(
#     Blank=True,
# #     hpad=0.1,
# #     wpad=0.1,
#     FontSize = 5.5,
#     Width=inches2mm(3.25),
#     subplot=111,
# #     axes_aspect="equal",
# #     Aspect=4.0/3.0,
# #     axes_aspect="equal",
# )

# AddContour(OverallMainPlot, 111, "Temp",

# OverallMainPlot = AddContour(
#                                 False,
#                                 111,
#                                 "Temp",
#                                 FontSize = FontSize,
#                                 Width = MP.inches2mm(3.5),
#                                 Aspect=4.0/3.0,
#                                 axes_aspect="equal",
#                                 hpad=0.0,
#                                 wpad=0.0,
#                             )

# ImgFileName = "Contour-Multi.pdf"

# OverallMainPlot.SaveFig(
#     ImgFileName,
#     bbox_inches = 'tight',
#     antialiased=True
# )

# img = WImage(resolution=450, filename=ImgFileName)
print "Really Done"
# img
if ShowPlots:
    plt.show()

/home/vatir/anaconda3/envs/DataViewer27/lib/python2.7/site-packages/numpy/lib/shape_base.py:422: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


Done
Done
Done
Done
Done
[0.86889434341331251, 0.98121877345141395]
[0.89135922942093282, 0.91382411542855313, 0.93628900143617333, 0.95875388744379364]
Done


/home/vatir/anaconda3/envs/DataViewer27/lib/python2.7/site-packages/matplotlib/font_manager.py:1288: UserWarning: findfont: Font family [u'cursive'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


Really Done


In [18]:
DataLocations["M"]

(1e-08, 1e-08)

In [19]:
Key = "M"
print DataLocations[Key][1] < 9e-11
print DataLocations[Key][1] > 9e-4
print DataLocations[Key][0] < 9e-11
print DataLocations[Key][0] > 9e-4

False
False
False
False


In [20]:
"{{{:.0f}}}".format(10.0)

'{10}'

In [21]:
def AddGradLinePlot(
    MainPlot,
    PlotIndex,
    DataSet,
    Width = MP.inches2mm(3.25),
    FontSize = 5.5,
    Aspect= 4.0/3.0,
    hpad=0.2,
    wpad=0.40,
    ShiftRight = 0.0,
    Type = "Conc", # "Conc" or "Time"
    Class = "Species", # "Species" or "Asm"
    ):
    
    import matplotlib.pyplot as plt
    from matplotlib.collections import LineCollection
    import matplotlib.cm as cm

    YAxis = FullOutArray[::-1,::-1,:,:,:]
    XMin = 0.0
    XMax = 0.0
    YMin = 0.0
    YMax = 0.0
    
    Key = DataIndices[DataSet]
    if Type == "Conc":
        TitleText = "Initial Monomer Concentration (M)"
        CData = YAxis[Key[0],Key[1],:, TimeTarget,:]
        XAxis = A0Range
        if np.max(CData) > YMax:
            YMax = np.max(CData)
        if np.max(CData) < YMin:
            YMin = np.min(CData)

    elif Type == "Time":
        TitleText = "Time (s)"
        CData = YAxis[Key[0],Key[1],A0Target, :,:]
        XAxis = TimePoints
        if np.max(CData) > YMax:
            YMax = np.max(CData)
        if np.max(CData) < YMin:
            YMin = np.min(CData)

    if Class == "Asm":
        YLabel = "Assembly Pathway Index"
        Flux = np.zeros(len(SortedPathways.values()))
        FluxByX = np.zeros([len(XAxis),len(SortedPathways.values())])
        for i, CA0 in enumerate(XAxis):
            if Type == "Conc":
                P.UpdateConcentrations(CData[i,:], CA0)
            elif Type == "Time":
                P.UpdateConcentrations(CData[i,:], A0)
            else:
                return
            for k, Path in enumerate(SortedPathways.values()):
                Flux[k] = P.PathIndependentFlux(Path, Probability = True)
            Flux = Flux/np.sum(Flux)
            FluxByX[i] = Flux

        LineSet = FluxByX
#         print np.sum(FluxByX, 1).shape
#         print np.sum(FluxByX, 1)
    elif Class == "Species":
#         print np.sum(CData,1)
        YLabel = "Species Index"
        LineSet = CData
        
    if MainPlot:
        MF = MainPlot._MainFig
    else:
        MF = MainPlot

    MainPlot = MP.LinePlot(
        YAxisInPercent = False,
        Width=Width,
        PrimaryFig=MF,
        subplot=PlotIndex,
        Aspect=Aspect,
        hpad = hpad,
        wpad = wpad,
        FontSize = FontSize,
        OF = False,
    #         axes_aspect=axes_aspect,
    )

    def AddLine(Index, XAxis, Data, PlotAxis):
        L = [[[Data[i,Index],Index+1],[XAxis[i],Index+1]] for i in range(len(XAxis))]

#         ColorMap = cm.plasma
#         ColorMap = cm.Reds
#         ColorMap = cm.inferno
#         ColorMap = cm.gray_r
#         ColorMap = cm.gray
        ColorMap = cm.binary
#         ColorMap = cm.plasma_r

        Line = LineCollection(
            L,
            cmap=ColorMap,
            norm=plt.Normalize(0, 1),
            rasterized=True,
        )
        Line.set_array(Data[:,Index])
        Line.set_linewidth(20)

        # add LineCollection to plot
        PlotAxis.add_collection(Line)

    NumberOfLines = LineSet.shape[1]
    for Index in range(NumberOfLines):
        AddLine(Index, XAxis, LineSet, MainPlot._Plot)

    # set plotting range
    MainPlot._Plot.set_xlim(np.min(XAxis), np.max(XAxis))
    MainPlot._Plot.set_xscale("log")
    MainPlot._Plot.set_ylim(0.5,NumberOfLines+0.5)

    # YTicks = MainPlot._Plot.yaxis.get_major_ticks()
    # for Index in np.arange(1, len(YTicks)):
    #     YTicks[Index].label1On = Index
    XTicks = MainPlot._Plot.xaxis.get_major_ticks()
    for Index in np.arange(0, len(XTicks),2):
        XTicks[Index].label1On = ""
    plt.yticks(np.arange(1.0,NumberOfLines+1.0,1.0))
    YTicks = MainPlot._Plot.yaxis.get_major_ticks()
    for Index in np.arange(0, len(YTicks),2):
        YTicks[Index].label1On = ""
    MainPlot.XLabel(TitleText)
    MainPlot.YLabel(YLabel)
    MainPlot._Plot.tick_params(
    #                 which = 'major',
            labelsize=FontSize*0.8,
    #                 pad = 1.5, # How far are the labels from the mark
    )

    Title = mlines.Line2D([], [],
                          color=ColorList[DataSet],
                          marker='o',
                          linestyle='None',
                          markersize=10,
                         )
    plt.legend(
        [Title],
        [r"Inter-Ring K$_d$ $10^{{{:.0f}}}$ (M) : Intra-Ring K$_d$ $10^{{{:.0f}}}$ (M)".format(*np.log10(DataLocations[DataSet]))],
        numpoints=1,
        bbox_to_anchor=(0., 1.0, 1., .08),
        loc=9,
        ncol=1,
    #         mode="expand",
        borderaxespad=0.,
        frameon=False,
        fontsize=FontSize,
    )
    print "Done: AddGradLinePlot"
    return MainPlot

In [22]:
OverallMainPlot = MPLGPU(
    Blank=True,
#     hpad=0.1,
#     wpad=0.1,
#     FontSize = FontSize,
    Width=MP.inches2mm(6.5),
    subplot=321,
#     axes_aspect="equal",
    Aspect=16.0/12.0,
    axes_aspect=16.0/12.0,
    **Settings
)

Type = "Conc"
Class = "Asm"
AddGradLinePlot(OverallMainPlot, gs1[0,0], "SW", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[0,1], "SS", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[1,1], "M", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,0], "WW", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,1], "WS", Type=Type, Class=Class,  **Settings)
AddLineFracVs(OverallMainPlot, gs1[1,0], CDataSet, TimeOrConc = Type, **Settings)

ImgFileName = "{}-{}-Multi-{}.pdf".format(Type, Class, CDataSet)
ImgFileName = "".join(ImgFileName.split())
OverallMainPlot.SaveFig(
    ImgFileName,
    bbox_inches = 'tight',
    antialiased=True
)
print "Really Done"
if ShowPlots:
    plt.show()

Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done
Really Done


In [23]:
OverallMainPlot = MPLGPU(
    Blank=True,
#     hpad=0.1,
#     wpad=0.1,
#     FontSize = FontSize,
    Width=MP.inches2mm(6.5),
    subplot=321,
#     axes_aspect="equal",
    Aspect=16.0/12.0,
    axes_aspect=16.0/12.0,
    **Settings
)

Type = "Time"
Class = "Asm"
AddGradLinePlot(OverallMainPlot, gs1[0,0], "SW", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[0,1], "SS", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[1,1], "M", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,0], "WW", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,1], "WS", Type=Type, Class=Class,  **Settings)
AddLineFracVs(OverallMainPlot, gs1[1,0], CDataSet, TimeOrConc = Type, **Settings)

ImgFileName = "{}-{}-Multi-{}.pdf".format(Type, Class, CDataSet)
ImgFileName = "".join(ImgFileName.split())
OverallMainPlot.SaveFig(
    ImgFileName,
    bbox_inches = 'tight',
    antialiased=True
)
print "Really Done"
if ShowPlots:
    plt.show()

Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done
Really Done


In [24]:
OverallMainPlot = MPLGPU(
    Blank=True,
#     hpad=0.1,
#     wpad=0.1,
#     FontSize = FontSize,
    Width=MP.inches2mm(6.5),
    subplot=321,
#     axes_aspect="equal",
    Aspect=16.0/12.0,
    axes_aspect=16.0/12.0,
    **Settings
)

Type = "Conc"
Class = "Species"
AddGradLinePlot(OverallMainPlot, gs1[0,0], "SW", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[0,1], "SS", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[1,1], "M", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,0], "WW", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,1], "WS", Type=Type, Class=Class,  **Settings)
AddLineFracVs(OverallMainPlot, gs1[1,0], CDataSet, TimeOrConc = Type, **Settings)

ImgFileName = "{}-{}-Multi-{}.pdf".format(Type, Class, CDataSet)
ImgFileName = "".join(ImgFileName.split())
OverallMainPlot.SaveFig(
    ImgFileName,
    bbox_inches = 'tight',
    antialiased=True
)
print "Really Done"
if ShowPlots:
    plt.show()

Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done
Really Done


In [25]:
OverallMainPlot = MPLGPU(
    Blank=True,
#     hpad=0.1,
#     wpad=0.1,
#     FontSize = FontSize,
    Width=MP.inches2mm(6.5),
    subplot=321,
#     axes_aspect="equal",
    Aspect=16.0/12.0,
    axes_aspect=16.0/12.0,
    **Settings
)

Type = "Time"
Class = "Species"
AddGradLinePlot(OverallMainPlot, gs1[0,0], "SW", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[0,1], "SS", Type=Type, Class=Class, **Settings)
AddGradLinePlot(OverallMainPlot, gs1[1,1], "M", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,0], "WW", Type=Type, Class=Class,  **Settings)
AddGradLinePlot(OverallMainPlot, gs1[2,1], "WS", Type=Type, Class=Class,  **Settings)
AddLineFracVs(OverallMainPlot, gs1[1,0], CDataSet, TimeOrConc = Type, **Settings)

ImgFileName = "{}-{}-Multi-{}.pdf".format(Type, Class, CDataSet)
ImgFileName = "".join(ImgFileName.split())
OverallMainPlot.SaveFig(
    ImgFileName,
    bbox_inches = 'tight',
    antialiased=True
)
print "Really Done"
if ShowPlots:
    plt.show()

Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done: AddGradLinePlot
Done
Really Done
